In [5]:
from katabatic.models.TableGAN import TableGANAdapter, TableGAN, preprocess_data, postprocess_data
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier, XGBRegressor
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# Initialize the adapter with a specific privacy setting
tablegan_adapter = TableGANAdapter(type='continuous', privacy_setting='high')
data_path = 'data/loan/Bank_Personal_Loan.csv'
df = pd.read_csv(data_path)
df['CCAvg']=df['CCAvg'].str.replace("/",".")
df["CCAvg"] = df["CCAvg"].astype(str).astype(float)
negative_count = (df['Experience'] < 0).sum()
print(negative_count)
df["Experience"] =abs(df["Experience"])
df.info()
print(df[:10])


52
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   ZIP Code            5000 non-null   int64  
 5   Family              5000 non-null   int64  
 6   CCAvg               5000 non-null   float64
 7   Education           5000 non-null   int64  
 8   Mortgage            5000 non-null   int64  
 9   Personal Loan       5000 non-null   int64  
 10  Securities Account  5000 non-null   int64  
 11  CD Account          5000 non-null   int64  
 12  Online              5000 non-null   int64  
 13  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 547.0 KB
   ID  Age  Experience  Income  ZIP Code  Family  CCA

In [6]:
x = df.drop("Personal Loan", axis = 1)
y = df["Personal Loan"].values.reshape(-1,1)
print("X shape is", x.shape)
print()
print("y shape is", y.shape)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, train_size=0.8, random_state=90049, shuffle=True, stratify=y)

X shape is (5000, 13)

y shape is (5000, 1)


In [7]:
tablegan_adapter.fit(x_train, y_train, epochs=100, batch_size=64)

---FIT TableGAN Model with high privacy setting


d:\Anaconda3\envs\py39\lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\Anaconda3\envs\py39\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


---Initialise TableGAN Model


d:\Anaconda3\envs\py39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 10/100: [D loss: -1.5253] [G loss: 0.6486] [C loss: 0.0942]
Epoch 20/100: [D loss: -0.8207] [G loss: 0.3043] [C loss: 0.0674]
Epoch 30/100: [D loss: -0.5415] [G loss: 0.3467] [C loss: 0.0591]
Epoch 40/100: [D loss: -0.5820] [G loss: 0.0892] [C loss: 0.0531]
Epoch 50/100: [D loss: -0.4907] [G loss: 0.6886] [C loss: 0.0492]
Epoch 60/100: [D loss: -0.4447] [G loss: 0.3717] [C loss: 0.0479]
Epoch 70/100: [D loss: -0.4322] [G loss: 0.4619] [C loss: 0.0440]
Epoch 80/100: [D loss: -0.4153] [G loss: 0.3388] [C loss: 0.0422]
Epoch 90/100: [D loss: -0.4284] [G loss: 0.4799] [C loss: 0.0413]
Epoch 100/100: [D loss: -0.4297] [G loss: 0.5591] [C loss: 0.0383]


In [8]:
# Generate synthetic data
synthetic_data = tablegan_adapter.generate(size=4000)
#synthetic_data = tablegan_adapter.generate(size=100)

---Generate from TableGAN Model


In [9]:
synthetic_df = pd.DataFrame(synthetic_data)
x_sync_train = synthetic_df.drop(synthetic_df.columns[-1],axis=1).values
y_sync_train = synthetic_df.iloc[ :, -1:].values
print(f"{type(x_sync_train)}, {x_sync_train.shape}")
print(f"{type(y_sync_train)}, {y_sync_train.shape}")

<class 'numpy.ndarray'>, (4000, 13)
<class 'numpy.ndarray'>, (4000, 1)


In [ ]:
# TSTR (train synthetic test real)
tstr_score_lr  = LogisticRegression().fit(x_sync_train, y_sync_train).score(x_test, y_test)
tstr_score_rf  = RandomForestClassifier().fit(x_sync_train, y_sync_train).score(x_test, y_test)
tstr_score_mlp = MLPClassifier().fit(x_sync_train, y_sync_train).score(x_test, y_test)
y_sync_train = LabelEncoder().fit_transform(y_sync_train)
xgbt_classifier = XGBClassifier(eval_metric='logloss')
tstr_score_xgbt = xgbt_classifier.fit(x_sync_train, y_sync_train).score(x_test, y_test)

# TRTR (train real test real)
trtr_score_lr  = LogisticRegression().fit(x_train, y_train).score(x_test, y_test)
trtr_score_rf  = RandomForestClassifier().fit(x_train, y_train).score(x_test, y_test)
trtr_score_mlp = MLPClassifier().fit(x_train, y_train).score(x_test, y_test)
xgbt_classifier = XGBClassifier(eval_metric='logloss', use_label_encoder=True)
trtr_score_xgbt = xgbt_classifier.fit(x_train, y_train).score(x_test, y_test)
df_evaluate = pd.DataFrame([
    ['TSTR', tstr_score_lr, tstr_score_rf, tstr_score_mlp, tstr_score_xgbt],
    ['TRTR', trtr_score_lr,trtr_score_rf,trtr_score_mlp, trtr_score_xgbt]
], columns=['Evaluated Item', 'LR', 'RF', 'MLP', 'XGBT'])
print(df_evaluate)

d:\Anaconda3\envs\py39\lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\Anaconda3\envs\py39\lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
d:\Anaconda3\envs\py39\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
d:\Anaconda3\envs\py39\lib\site-packages\s

  Evaluated Item     LR     RF    MLP   XGBT
0           TSTR  0.905  0.921  0.904  0.970
1           TRTR  0.906  0.991  0.904  0.991


"\nimport numpy as np\n# evaluate\n#ohe = OneHotEncoder(handle_unknown='ignore')\nohe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)\nlbe = LabelEncoder()\n\n# TRTR\nx_train_ohe = ohe.fit_transform(x_train)\nx_test_ohe = ohe.transform(x_test)\ny_train_lbe = lbe.fit_transform(y_train)\ny_test_lbe = lbe.transform(y_test)\ntrtr_score_lr  = LogisticRegression().fit(x_train_ohe, y_train_lbe).score(x_test_ohe, y_test_lbe)\ntrtr_score_rf  = RandomForestClassifier().fit(x_train, y_train_lbe).score(x_test, y_test_lbe)\ntrtr_score_mlp = MLPClassifier().fit(x_train_ohe, y_train_lbe).score(x_test_ohe, y_test_lbe)\nxgbt_classifier = XGBClassifier(use_label_encoder=False, eval_metric='logloss')\ntrtr_score_xgbt = xgbt_classifier.fit(x_train_ohe, y_train_lbe).score(x_test_ohe, y_test_lbe)\n\n# TSTR\nx_train_ohe = ohe.fit_transform(x_sync_train)\nx_test_ohe = ohe.fit_transform(x_test)\ny_train_lbe = lbe.fit_transform(y_sync_train)\ny_test_lbe = lbe.fit_transform(y_test)\ntstr_s